In [1]:
# Loading in the modules and gene_human_table
from connection import DB_connect # This only works because there is a directory called connection with DB_connect.py in it
import pandas as pd
import datetime
import getpass
import pagerload_signor
data = pagerload_signor.extract_data()

### Gene table ###
GENE_HUMAN_TABLE = data.GET_HUMAN_GENE()

#import sys, os
#import re
#import math
#import codecs
#from scipy.stats import hypergeom

In [2]:
# account = 'nick95'
# pwd = 'NickSumpter'
account = input()
pwd = getpass.getpass ('Write your password here:')

nick95
Write your password here:········


In [2]:
### SIGNOR data extract, transform, load
from SIGNOR import signor
SIGNOR = signor.SIGNOR()

pd_signor_pathway_map = SIGNOR.GET_HUMAN_SIGNOR_PATHWAY()

pd_signor_pathway_map = pd_signor_pathway_map.merge(
    GENE_HUMAN_TABLE[['GENEID','SYMBOL']], left_on = ['SYMBOL'], right_on = ['SYMBOL'])

pd_signor_description_map = SIGNOR.GET_SIGNOR_DETAILS()

In [3]:
### PAG data generation from SIGNOR
RECORD_DATE = datetime.datetime(2022, 4, 26, 0, 0)
three_letter = "WAG"

data.SET_ATTRIBUTES(three_letter = three_letter,
                    PAG_SOURCE = 'SIGNOR',
                    PAG_TYPE = 'P',
                    RECORD_DATE = RECORD_DATE,
                    STATUS = 'Published',
                    CURATOR = 'Zongliang Yue',
                    CURATOR_CONTACT = 'zongyue@uab.edu')

data.GENERATE_PAG_SIGNOR(pd_signor_pathway_map = pd_signor_pathway_map,
                         pd_signor_description_map = pd_signor_description_map)

In [4]:
data.check_data(0)

('WAG016472', 'Aspartate and asparagine metabolism', 14, 'Homo sapiens', '', 'https://signor.uniroma2.it/getPathwayData.php?description', '', '', '', 'Published', None, None, '', '', 0, '', 3.0, '', 'SIGNOR-AAAM', datetime.datetime(2022, 4, 26, 0, 0))
('WAG016472', 468, 'ATF4')
('WAG016472', 'SIGNOR')
('WAG016472', 'P')


In [5]:
data.insert_data()

In [8]:
### Checking data in PAGER3 ###
# PAGER3 connection:
oraDB = DB_connect.DB_connect()
oraDB.setID('infoinst-02.rc.uab.edu', '1521', 'BIODB.RC.UAB.EDU')
oraDB.getConn(account, pwd)

# Retrieve the existing PAGs from PAGER3
sql = 'SELECT * FROM nick95.vw_ds_gs_all'

ds_gs_all = oraDB.getValue(sql, oraDB.db)
ds_gs_all = pd.DataFrame(ds_gs_all)
oraDB.close()

ds_gs_all.columns=['GS_ID', 'NAME', 'GS_SIZE', 'ORGANISM', 'DESCRIPTION',
                   'LINK', 'REFERENCE', 'PUBMED_ID', 'CATEGORY', 'STATUS',
                   'CURATOR', 'CURATOR_CONTACT', 'CONTRIBUTED_BY', 'FLAG',
                   'COCO_V2', 'GO_FLAG', 'VERSION', 'REF_OLD_ID',
                   'ORIGINAL_ID', 'RECORD_DATE', 'SOURCE', 'TYPE', 'MULTI_N']

print(ds_gs_all[ds_gs_all['SOURCE'].isin(['SIGNOR'])])

            GS_ID                                 NAME  GS_SIZE      ORGANISM  \
106342  WAG016474                            Alzheimer       22  Homo sapiens   
106876  WAG016472  Aspartate and asparagine metabolism       14  Homo sapiens   
106877  WAG016473                         Adipogenesis       23  Homo sapiens   
107093  WAG016480                    Citric acid cycle        7  Homo sapiens   
107408  WAG016475                        Axon guidance       20  Homo sapiens   
...           ...                                  ...      ...           ...   
129207  WAG016554                  Toll like receptors       18  Homo sapiens   
129208  WAG016555                       VEGF Signaling       17  Homo sapiens   
129209  WAG016556                      Vitamin-K cycle       11  Homo sapiens   
129210  WAG016557                        WNT Signaling       24  Homo sapiens   
129211  WAG016558         WNT Signaling and Myogenesis       24  Homo sapiens   

                           

In [9]:
# Code for getting the modules to load correctly
#sys.path.append('/home/nicks95/tools/miniconda3')
#!source activate /home/nicks95/tools/miniconda3/envs/py38
#!pip install cx_Oracle
#!pip freeze

In [ ]:
# Everything below here should relate to the nCoCo scores

In [4]:
### Triangle calculation ###
# Retrieve pag_to_gene table #
oraDB.setID('infoinst-02.rc.uab.edu', '1521', 'BIODB.RC.UAB.EDU')
oraDB.getConn(account, pwd)

sql = 'select * from nick95.ds_gs_all_gene'
#sql += ' and RECORD_DATE = \'24-JUN-20\''

ds_gs_all_gene = oraDB.getValue(sql, oraDB.db)
ds_gs_all_gene = pd.DataFrame(ds_gs_all_gene)
oraDB.close()

ds_gs_all_gene.columns = ['GS_ID', 'GENE_ID', 'GENE_SYM']
ds_gs_all_gene.head(2)

,GS_ID,GENE_ID,GENE_SYM
0,FEX003015,23032,USP33
1,FEX003015,7336,UBE2V2


In [346]:
def createListSql(gene):
    ### create list in sql ###
    if len(gene) == 0:
        gn = "'" + str(gene) + "'"
    else:
        gnFirst = str(gene[0])
        gn = "'" + gnFirst + "'"
        for i in range(1, len(gene)):
            gn = gn + ", '" + str(gene[i]) + "'"
    ### create list >1000 in sql ###
    if len(gene) >= 1000:
        print('more than 1000')
        num_partitions = int(len(gene) / 999) + 1
        parts = []
        for i in range(1, num_partitions + 1):
            if(i < num_partitions):
                limit = 999
            else: #last partition
                limit = len(gene) - (999 * (num_partitions - 1))
            genes_separated_by_commas = ""
            for j in range(0, limit):
                genes_separated_by_commas = genes_separated_by_commas + "'" + gene[j + (999 * (i - 1))] + "', "
            genes_separated_by_commas = genes_separated_by_commas[0:-1]
            string = 'SELECT COLUMN_VALUE SYM FROM TABLE(SYS.DBMS_DEBUG_VC2COLL(' + genes_separated_by_commas + '))'
            parts.append(string)
        temp_gene_table = ""
        for i in range(0, len(parts)):
            if(i < len(parts) - 1):
                temp_gene_table = temp_gene_table + parts[i] + ' UNION '
            else:
                temp_gene_table = temp_gene_table + parts[i]
    else:
        temp_gene_table = 'SELECT COLUMN_VALUE SYM FROM TABLE(SYS.DBMS_DEBUG_VC2COLL(' + gn + '))'
    return(temp_gene_table)

In [ ]:
oraDB.setID('watson.informatics.uab.edu', '1521', 'BIODB.RC.UAB.EDU')
oraDB.getConn(account, pwd)

sql = 'select * from nick95.ds_gs_all, nick95.ds_gs_all_source where nick95.ds_gs_all.gs_id in (' + createListSql(PAG_IDs) + ') and nick95.ds_gs_all.gs_id = nick95.ds_gs_all_source.gs_id'
#sql += ' and RECORD_DATE = \'24-JUN-20\''

res_old = oraDB.getValue(sql, oraDB.db)
res_old_PD = pd.DataFrame(res_old)
oraDB.close()

GS_ID_pd = res_old_PD[[0]]

In [ ]:
### Retrieving gene-gene interaction information from HAPPI ###
oraDB.setID('infoinst-02.rc.uab.edu', '1521', 'BIODB.RC.UAB.EDU')
oraDB.getConn(account, pwd)

sql = 'select distinct SYM_A, SYM_B from HAPPI where H_SCORE >= 0.45 and SYM_A != SYM_B'

happi = oraDB.getValue(sql, oraDB.db)
HAPPI_PD = pd.DataFrame(happi)
oraDB.close()

HAPPI_PD.columns = ['SYM_A', 'SYM_B']
HAPPI_PD.head(2)

In [ ]:
### find the gene number of PAGs ###
GS_ID_pd.columns = ['GS_ID']
DS_GS_ALL_GENE_PD = DS_GS_ALL_GENE_PD.merge(GS_ID_pd, left_on = 'GS_ID', right_on = 'GS_ID', how = 'inner')

gene_count_pd = DS_GS_ALL_GENE_PD.groupby(['GS_ID']).agg(['count'])
gene_count_pd = gene_count_pd[['GENE_ID']]
gene_count_pd.columns = ['size']

In [ ]:
# PPI pairs calculation #
GS_IDs = GS_ID_pd['GS_ID'].unique()
ppi_pd = pd.DataFrame()
GS_IDs.shape[0]
a = 0
for GS_ID in GS_IDs.tolist():
    a += 1
    print("%s:%s" %(a, GS_ID))
    GS_TARGETED = DS_GS_ALL_GENE_PD[DS_GS_ALL_GENE_PD['GS_ID'].isin([GS_ID])]
    GS_TARGETED = DS_GS_ALL_GENE_PD[DS_GS_ALL_GENE_PD['GS_ID'].isin([GS_ID])]
    GS_TARGETED_A_IN = GS_TARGETED.merge(HAPPI_PD, left_on = 'GENE_SYMBOL', right_on = 'SYM_A')
    GS_TARGETED_B_IN = GS_TARGETED.merge(HAPPI_PD, left_on = 'GENE_SYMBOL', right_on = 'SYM_B')
    GS_TARGETED_AB_IN = GS_TARGETED_B_IN.merge(GS_TARGETED_A_IN, left_on = ['GS_ID', 'SYM_A', 'SYM_B'], right_on = ['GS_ID', 'SYM_A', 'SYM_B'])
    GS_TARGETED_AB_IN = GS_TARGETED_AB_IN[['GS_ID', 'SYM_A', 'SYM_B']]
    GS_TARGETED_AB_IN.columns = ['GS_ID', 'SYM_A', 'SYM_B']
    PPI_count_pd = GS_TARGETED_AB_IN.groupby(['GS_ID']).agg(['count'])
    PPI_count_pd = PPI_count_pd[['SYM_A']]
    PPI_count_pd.columns = ['PPI']
    res_ppi_pd = gene_count_pd.merge(PPI_count_pd, left_on = ['GS_ID'], right_on = ['GS_ID'])
    ppi_pd=ppi_pd.append(res_ppi_pd)

In [ ]:
ppi_pd.to_csv('ppi_count_KEGG_WIKI_REACTOME_2021.txt', sep = "\t")

In [ ]:
GS_IDs = GS_ID_pd['GS_ID'].unique()
triangle_pd = pd.DataFrame()
GS_IDs.shape[0]
a = 0
for GS_ID in GS_IDs.tolist():
    a += 1
    print("%s:%s" %(a, GS_ID))
    GS_TARGETED = DS_GS_ALL_GENE_PD[DS_GS_ALL_GENE_PD['GS_ID'].isin([GS_ID])]
    GS_TARGETED_A_IN = GS_TARGETED.merge(HAPPI_PD, left_on = 'GENE_SYMBOL', right_on = 'SYM_A')
    GS_TARGETED_B_IN = GS_TARGETED.merge(HAPPI_PD, left_on = 'GENE_SYMBOL', right_on = 'SYM_B')
    GS_TARGETED_AB_IN = GS_TARGETED_B_IN.merge(GS_TARGETED_A_IN,left_on = ['GS_ID', 'SYM_A', 'SYM_B'], right_on = ['GS_ID', 'SYM_A', 'SYM_B'])
    GS_TARGETED_AB_IN = GS_TARGETED_AB_IN[['GS_ID', 'SYM_A', 'SYM_B']]
    GS_TARGETED_AB_IN.columns = ['GS_ID', 'SYM_A', 'SYM_B']
    GS_TARGETED_AB_IN_1 = GS_TARGETED_AB_IN.merge(GS_TARGETED_AB_IN, left_on = ['SYM_A'], right_on = ['SYM_B'])
    GS_TARGETED_AB_IN_2 = GS_TARGETED_AB_IN_1.merge(GS_TARGETED_AB_IN, left_on = ['SYM_B_x', 'SYM_A_y'], right_on = ['SYM_B', 'SYM_A'])
    triangle_count_pd = GS_TARGETED_AB_IN_2.groupby(['GS_ID']).agg(['count'])
    triangle_count_pd = triangle_count_pd[['SYM_A_x']]
    
    
    triangle_count_pd.columns = ['triangle']
    res_pd = gene_count_pd.merge(triangle_count_pd, left_on = ['GS_ID'], right_on = ['GS_ID'])
    triangle_pd=triangle_pd.append(res_pd)


In [ ]:
triangle_pd.to_csv('triangle_count_KEGG_WIKI_REACTOME_2021.txt', sep = "\t")

In [347]:
### nCoCo score updates
# nCoCo score -- CoI score
def create_view_CoI(restriction,table_name):
    SQLHAPPI3STAR = ' (select distinct SYM_A, SYM_B from HAPPI where H_SCORE >= 0.45 and SYM_A != SYM_B) '
    SQL = 'create or replace view vw_tmp_gs_happi_interaction_ab as '
    SQL += 'select R2.GS_ID GS_ID, R1.SYM_B Mol_X_ID, R1.SYM_A Mol_IN_ID '
    SQL += 'from ' + SQLHAPPI3STAR + ' R1, ' + table_name + ' R2 '
    SQL += 'where R2.GENE_SYM = R1.SYM_A '+ restriction
    SQL += ' union '
    SQL += 'select R2.GS_ID GS_ID, R1.SYM_A Mol_X_ID, R1.SYM_B Mol_IN_ID '
    SQL += 'from ' + SQLHAPPI3STAR + ' R1, ' + table_name + ' R2 '
    SQL += 'where R2.GENE_SYM = R1.SYM_B ' + restriction
    SQL
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    SQL = 'create or replace view vw_tmp_gs_happi_inter_ab_in as select X.GS_ID, X.MOL_X_ID IN_B_ID, X.Mol_IN_ID IN_A_ID from vw_tmp_gs_happi_interaction_ab X , '+table_name+' Y where Y.GS_ID= X.GS_ID and Y.gene_sym = X.Mol_X_ID'
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    SQL = 'create or replace view vw_tmp_gs_happi_inter_ab_out as select gs_id, mol_in_id, mol_x_id from  vw_tmp_gs_happi_interaction_ab minus select gs_id, in_a_id, in_b_id from vw_tmp_gs_happi_inter_ab_in minus select gs_id, in_b_id, in_a_id from vw_tmp_gs_happi_inter_ab_in'
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    SQL = 'create or replace view vw_tmp_gs_happi_inter_in_num as select gs_id, count(*) num from vw_tmp_gs_happi_inter_ab_in group by gs_id'
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    SQL = 'create or replace view vw_tmp_gs_happi_inter_out_num as select gs_id, count(*) num from vw_tmp_gs_happi_inter_ab_out group by gs_id'
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    SQL = 'create or replace view vw_tmp_gs_happi_inter_out_gn_n as select gs_id, count(distinct(mol_x_id)) num from vw_tmp_gs_happi_inter_ab_out group by gs_id'
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    SQL = 'create or replace view vw_tmp_gs_happi_inter_num as select vw_tmp_gs_happi_inter_in_num.gs_id, vw_tmp_gs_happi_inter_in_num.num in_num, vw_tmp_gs_happi_inter_out_num.num out_num from vw_tmp_gs_happi_inter_in_num inner join vw_tmp_gs_happi_inter_out_num on vw_tmp_gs_happi_inter_in_num.gs_id=vw_tmp_gs_happi_inter_out_num.GS_ID'
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    SQL = 'create or replace view vw_tmp_gs_happi_inter_num_a as select X.gs_id, X.in_num, X.out_num, Y.num out_g_num from vw_tmp_gs_happi_inter_num X inner join vw_tmp_gs_happi_inter_out_gn_n Y on X.gs_id=Y.GS_ID'
    oraDB.execSql(SQL, oraDB.db)
    oraDB.commit(oraDB.db)
    
def get_CoI_file(table_name):
    SQL = 'select X.GS_ID, Y.GS_SIZE, X.in_num/2 INSIDE_NUM, X.out_num OUTSIDE_NUM, X.out_g_num outside_g_num from vw_tmp_gs_happi_inter_num_a X, (select gs_id, count(*) GS_SIZE from '+table_name+' group by gs_id) Y where X.GS_ID=Y.GS_ID'
    print(SQL)
    sta_file = oraDB.getValue(SQL, oraDB.db)
    sta_file_PD = pd.DataFrame(sta_file)
    sta_file_PD = sta_file_PD.rename(columns = {0:'GS_ID', 1:'GS_SIZE', 2:'INSIDE_NUM', 3:'OUTFLOW_NUM', 4:'OUTSIDE_INTERACTOR'})
    return sta_file_PD

# nCoCo score -- CoT score
def get_CoT_file(table_name, restriction1):
    SQL = "select R5.gs_id, R6.GS_SIZE, R5.TRIANGLE from "
    SQL += "( "
    SQL += "select R3.gs_id, count(*) TRIANGLE "
    SQL += "from "
    SQL += "( "
    SQL += "select R1.gs_id, R1.IN_A_ID, R1.IN_B_ID INTER, R2.IN_B_ID "
    SQL += "from vw_tmp_gs_happi_inter_ab_in R1, vw_tmp_gs_happi_inter_ab_in R2 "
    SQL += "where R1.IN_B_ID = R2.IN_A_ID and R1.GS_ID = R2.GS_ID "
    SQL += ") "
    SQL += "R3, vw_tmp_gs_happi_inter_ab_in R4 "
    SQL += "where R3.IN_A_ID = R4.IN_A_ID and R3.IN_B_ID = R4.IN_B_ID and R3.gs_id = R4.GS_ID group by R3.gs_id"
    SQL += ") R5, (select gs_id, count(*) GS_SIZE from "+"(select * from ds_gs_all_gene where " + restriction1 + ")" + " group by gs_id) R6 "
    SQL += "where R5.gs_id = R6.gs_id"
    
    print(SQL)
    triangle = oraDB.getValue(SQL, oraDB.db)
    #triangle
    triangle_PD = pd.DataFrame(triangle)
    triangle_PD = triangle_PD.rename(columns = {0:'GS_ID', 1:'GS_SIZE', 2:'TRIANGLE'})
    return triangle_PD


In [349]:
# run CoCohypergeo script in "PAGER/calculation" instead #
### 3. bioRxiv PAG_IDs nCoCo score calculation ###
# 1. check the existing PAGs of PAGER 2.0 in PAGER-CoV
# PAGER-CoV connection:
oraDB = DB_connect.DB_connect()
account = ''
pwd = ''
oraDB = DB_connect.DB_connect()
oraDB.setID('infoinst-02.rc.uab.edu','1521','BIODB.RC.UAB.EDU')
oraDB.getConn(account,pwd)
# CoCo calculation based on CoI and CoT
restriction = 'AND GS_ID in (' + createListSql(PAG_IDs) +')'
#restriction
restriction1 = ' GS_ID in (' + createListSql(PAG_IDs)+')'
#restriction1
table_name='ds_gs_all_gene'
create_view_CoI(restriction,table_name)

# run CoI
sta_file_PD = get_CoI_file(table_name)
# sta_PAGER3_KEGG_WIKIPATHWAY_2021
sta_file_PD.to_csv('sta_PAGER3_Reactome_2021.txt', index = False, sep = "\t")



more than 1000
more than 1000
select X.GS_ID, Y.GS_SIZE, X.in_num/2 INSIDE_NUM, X.out_num OUTSIDE_NUM, X.out_g_num outside_g_num from vw_tmp_gs_happi_inter_num_a X, (select gs_id, count(*) GS_SIZE from ds_gs_all_gene group by gs_id) Y where X.GS_ID=Y.GS_ID


In [ ]:
# run CoT
triangle_PD = get_CoT_file(table_name,restriction1)
triangle_PD.to_csv('triangle_PAGER3_KEGG_WIKIPATHWAY_2021.txt', index = False, sep = "\t")

select R5.gs_id, R6.GS_SIZE, R5.TRIANGLE from ( select R3.gs_id, count(*) TRIANGLE from ( select R1.gs_id, R1.IN_A_ID, R1.IN_B_ID INTER,R2.IN_B_ID from vw_tmp_gs_happi_inter_ab_in R1, vw_tmp_gs_happi_inter_ab_in R2 where R1.IN_B_ID=R2.IN_A_ID and R1.GS_ID=R2.GS_ID ) R3, vw_tmp_gs_happi_inter_ab_in R4 where R3.IN_A_ID = R4.IN_A_ID and R3.IN_B_ID=R4.IN_B_ID and R3.gs_id=R4.GS_ID group by R3.gs_id) R5, (select gs_id, count(*) GS_SIZE from (select * from ds_gs_all_gene where  GS_ID in (SELECT COLUMN_VALUE SYM FROM TABLE(SYS.DBMS_DEBUG_VC2COLL('WAG001707','WAG001708','WAG001709','WAG001787','WAG001788','WAG001789','WAG001790','WAG001855','WAG001856','WAG001857','WAG001858','WAG001921','WAG001922','WAG001923','WAG001924','WAG002033','WAG002034','WAG002035','WAG002036','WAG002037','WAG002038','WAG002039','WAG002040','WAG002041','WAG002042','WAG002043','WAG002044','WAG002045','WAG002046','WAG002047','WAG002048','WAG002049','WAG002050','WAG002051','WAG002052','WAG002053','WAG002054','WAG002055'

In [216]:
### pager3 get ds_gs_all_gene ###
# PAGER3 connection:
oraDB = DB_connect.DB_connect()
account = ''
pwd = ''
oraDB = DB_connect.DB_connect()
oraDB.setID('infoinst-02.rc.uab.edu','1521','BIODB.RC.UAB.EDU')
oraDB.getConn(account,pwd)

SQL = "select * from PAGER3.ds_gs_all_gene, "
SQL += "( "
SQL += createListSql(PAG_IDs)
SQL += ") r "
SQL += "where PAGER3.ds_gs_all_gene.gs_id=r.SYM"

print(SQL)
ds_gS_all_gene = oraDB.getValue(SQL,oraDB.db)
#triangle
pd_ds_gs_all_gene = pd.DataFrame(ds_gS_all_gene)
pd_ds_gs_all_gene = pd_ds_gs_all_gene.rename(columns={0:'GS_ID',1:'GENEID',2:'GENE_SYM'})

oraDB.close()

more than 1000
select * from PAGER3.ds_gs_all_gene, ( SELECT /*+ append parallel(16) */ COLUMN_VALUE SYM FROM TABLE(SYS.DBMS_DEBUG_VC2COLL('WAG014907','WAG014908','WAG014909','WAG014910','WAG014911','WAG014912','WAG014913','WAG014914','WAG014915','WAG014916','WAG014917','WAG014918','WAG014919','WAG014920','WAG014921','WAG014922','WAG014923','WAG014924','WAG014925','WAG014926','WAG014927','WAG014928','WAG014929','WAG014930','WAG014931','WAG014932','WAG014933','WAG014934','WAG014935','WAG014936','WAG015139','WAG015140','WAG015141','WAG015142','WAG015143','WAG015144','WAG015145','WAG015146','WAG015147','WAG015148','WAG015149','WAG015150','WAG015151','WAG015152','WAG015153','WAG015154','WAG015155','WAG015156','WAG015157','WAG015158','WAG015159','WAG015160','WAG015161','WAG015162','WAG015163','WAG015164','WAG015165','WAG015351','WAG015352','WAG015353','WAG015354','WAG015355','WAG015356','WAG015357','WAG015358','WAG015359','WAG015360','WAG015361','WAG015362','WAG015363','WAG015364','WAG01536

In [211]:
pd_ds_gs_all_gene[['GS_ID','GENEID','GENE_SYM']].to_csv('PAG_GENE_11_14_2021.txt', index = False, sep = "\t")
pd_ds_gs_all_gene[['GS_ID','GENEID','GENE_SYM']].to_csv('/home/zongliang/PAGER/calculation/m-type/ADDING_PAG/input/PAGER3/PAG_GENE_11_14_2021.txt', index = False, sep = "\t")

In [217]:

pd_ds_gs_all_gene[['GS_ID','GENEID','GENE_SYM']].to_csv('/home/zongliang/PAGER/calculation/m-type/ADDING_PAG/input/PAGER3/NEW_PAG_GENE.txt', index = False, sep = "\t")

In [161]:
pd_ds_gs_all_gene[['GS_ID','GENEID','SYMBOL']].to_csv('inputgenepag_REACTOME_2021.csv', index = False, sep = "\t")

In [150]:
### warning: don't run it to delete the records
def createListSql(gene):
    ### create list in sql ###
    if len(gene) == 0:
        gn="'"+str(gene)+"'"
    else:
        gnFirst=str(gene[0])
        gn="'"+gnFirst+"'"
        for i in range(1,len(gene)):
            gn=gn+",'"+str(gene[i])+"'"
    #print("The gene numbers:"+str(len(gene)))
    ### create list >1000 in sql ###
    if len(gene)>=1000:
        print('more than 1000')
        num_partitions = int(len(gene)/999)+1
        parts=[]
        for i in range(1,num_partitions+1):
            if(i < num_partitions):
                limit = 999
            else:	#last partition
                limit = len(gene)-(999*(num_partitions-1))
            genes_separated_by_commas = ""
            for j in range(0,limit):
                genes_separated_by_commas = genes_separated_by_commas + "'" + gene[j+(999*(i-1))] + "',"
                #print $genes_separated_by_commas
            genes_separated_by_commas=genes_separated_by_commas[0:-1]
            string = 'SELECT /*+ append parallel(16) */ COLUMN_VALUE SYM FROM TABLE(SYS.DBMS_DEBUG_VC2COLL('+genes_separated_by_commas+'))'
            parts.append(string)      
        temp_gene_table = ""
        for i in range(0,len(parts)):
            if(i < len(parts)-1):
                temp_gene_table = temp_gene_table+parts[i]+' UNION '
            else:
                temp_gene_table = temp_gene_table+parts[i]
    else:
        temp_gene_table = 'SELECT /*+ append parallel(16) */ COLUMN_VALUE SYM FROM TABLE(SYS.DBMS_DEBUG_VC2COLL('+gn+'))'
    return(temp_gene_table)

### warning: don't run it to delete the records
oraDB.setID('infoinst-02.rc.uab.edu','1521','BIODB.RC.UAB.EDU')
oraDB.getConn(account, pwd)
sql = 'select * from ds_gs_all_source where source = \'Reactome_2021\'';
last_date_r = oraDB.getValue(sql, oraDB.db)
len(last_date_r)
last_date_r = pd.DataFrame(last_date_r)
oraDB.close()

13163

In [151]:
last_date_r

,0,1
0,WAG015613,Reactome_2021
1,WAG015614,Reactome_2021
2,WAG015615,Reactome_2021
3,WAG015616,Reactome_2021
4,WAG015617,Reactome_2021
...,...,...
13158,WAG005913,Reactome_2021
13159,WAG005914,Reactome_2021
13160,WAG005915,Reactome_2021
13161,WAG005916,Reactome_2021


In [31]:
### warning: don't run it to delete the records
oraDB.setID('infoinst-02.rc.uab.edu','1521','BIODB.RC.UAB.EDU')
oraDB.getConn(account,pwd)

# roll back and delete the previous members and insert the current members in ds_gs_all_gene
#PAG_input = pd.DataFrame(dataObj.PAG_input)
sql = 'delete from ds_gs_all where gs_id in ('+createListSql(last_date_r.loc[:,0].tolist())+')'
oraDB.execSql(sql,oraDB.db)
oraDB.commit(oraDB.db)
sql = 'delete from ds_gs_all_gene where gs_id in ('+createListSql(last_date_r.loc[:,0].tolist())+')'
oraDB.execSql(sql,oraDB.db)
oraDB.commit(oraDB.db)

sql = 'delete from ds_gs_all_type where gs_id in ('+createListSql(last_date_r.loc[:,0].tolist())+')'
oraDB.execSql(sql,oraDB.db)
oraDB.commit(oraDB.db)

sql = 'delete from ds_gs_all_source where gs_id in ('+createListSql(last_date_r.loc[:,0].tolist())+')'
oraDB.execSql(sql,oraDB.db)
oraDB.commit(oraDB.db)

oraDB.close()

In [163]:
### warning: don't run it to delete the records
oraDB.setID('infoinst-02.rc.uab.edu','1521','BIODB.RC.UAB.EDU')
oraDB.getConn(account,pwd)

sql = 'delete from ds_gs_all_gene_score where gs_id in (' + createListSql(last_date_r.loc[:,0].tolist()) + ')'
oraDB.execSql(sql,oraDB.db)
oraDB.commit(oraDB.db)

oraDB.close()

more than 1000
